In [ ]:
#pip installations
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
%pip install accelerate -U
%pip install datasets
%pip install evaluate
%pip install transformers
%pip install editdistance

## Step -1: Filter Parameters

### Import libraries and get embedding matrix

In [4]:
import torch
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F
import editdistance

# Load model
tokenizer = AutoTokenizer.from_pretrained("./bert_tokenizer")
model = AutoModel.from_pretrained("./models/BaseModel/checkpoint-27306")

# Get full embedding matrix


embedding_matrix = model.get_input_embeddings().weight  # shape: (vocab_size, hidden_size)
print("Embedding matrix shape:", embedding_matrix.shape)


device = torch.device("cpu" if torch.cuda.is_available() else "cpu")


Loading weights: 100%|██████████| 101/101 [00:00<00:00, 731.60it/s, Materializing param=encoder.layer.5.output.dense.weight]             
BertModel LOAD REPORT from: ./models/BaseModel/checkpoint-27306
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
pooler.dense.weight                        | MISSING    | 
pooler.dense.bias                          | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Embedding matrix shape: torch.Size([30000, 512])


## Step 1: Filter Out Irrelevant Tokens

In [12]:
#first 4981 tokens are unused or single characters
vocab_size = len(tokenizer.get_vocab())
vocab_tokens = [(i,tokenizer.decode([i])) for i in range(4981,vocab_size)]
print(vocab_tokens)

#filter out any tokens with numbers in them
filtered_tokens  = torch.tensor([idx for idx, token in vocab_tokens if not (any((char.isdigit() or char == '#')  for char in token) or len(token) < 3)])
print("Vocab size after removing tokens with numbers:", len(filtered_tokens))
print("Vocab size after removing single char tokens:", len(vocab_tokens))



[(4981, '🖕'), (4982, '##e'), (4983, '##a'), (4984, '##t'), (4985, '##h'), (4986, '##d'), (4987, '##l'), (4988, '##r'), (4989, '##i'), (4990, '##n'), (4991, '##m'), (4992, '##s'), (4993, '##I'), (4994, '##A'), (4995, '##F'), (4996, '##b'), (4997, '##y'), (4998, '##o'), (4999, '##k'), (5000, '##g'), (5001, '##v'), (5002, '##f'), (5003, '##u'), (5004, '##p'), (5005, '##ァ'), (5006, '##イ'), (5007, '##ナ'), (5008, '##ル'), (5009, '##フ'), (5010, '##ン'), (5011, '##タ'), (5012, '##ジ'), (5013, '##ー'), (5014, '##X'), (5015, '##c'), (5016, '##w'), (5017, '##é'), (5018, '##x'), (5019, '##B'), (5020, '##ǎ'), (5021, '##ć'), (5022, '##S'), (5023, '##O'), (5024, '##H'), (5025, '##9'), (5026, '##2'), (5027, '##5'), (5028, '##T'), (5029, '##V'), (5030, '##ā'), (5031, '##ö'), (5032, '##q'), (5033, '##4'), (5034, '##G'), (5035, '##E'), (5036, '##z'), (5037, '##D'), (5038, '##8'), (5039, '##0'), (5040, '##j'), (5041, '##R'), (5042, '##3'), (5043, '##P'), (5044, '##M'), (5045, '##š'), (5046, '##L'), (5047, '##ł

## Step 2: Calculate Similarity Matrix

In [13]:

embeddings = embedding_matrix[filtered_tokens].to(device)

# 1. Normalize embeddings to unit vectors
norm_embeddings = F.normalize(embeddings, p=2, dim=1)

# 2. Calculate the full Similarity Matrix
# Result is (N, N). For 20k tokens, this is ~800MB in float16
sim_matrix = torch.mm(norm_embeddings, norm_embeddings.t())

# 3. Mask the diagonal (Self-similarity is always 1.0)
n = sim_matrix.size(0)
diag_indices = torch.arange(n, device=sim_matrix.device)
sim_matrix[diag_indices, diag_indices] = -1.0  # Set to -1 so topk ignores them

# 4. Get the Top 40 closest neighbors for every single token
k_neighbors = 20
values, indices = torch.topk(sim_matrix, k=k_neighbors, largest=True, dim=1)

# 5. Find the 100 "Closest" pairs globally across the entire matrix
k_global = 10000
flat_values = values.view(-1)
print("Flat values shape:", flat_values.shape)
global_max_vals, global_max_idxs = torch.topk(flat_values, k=k_global * 2, largest=True)

# 6. Map back to token indices
# row_idx: The source word
# neighbor_idx: The similar word
row_indices = global_max_idxs // k_neighbors
neighbor_indices = indices.view(-1)[global_max_idxs]


print("global max vals shape:", global_max_vals.shape)



Flat values shape: torch.Size([312940])
global max vals shape: torch.Size([20000])


In [28]:
theta = 0.60
epsilon = 0.50

## Filter Pairs Based on Filter Parameters

In [30]:
seen_pairs = set()

for i in range(len(global_max_vals)):
    u, v = row_indices[i].item(), neighbor_indices[i].item()

    word1 = tokenizer.decode([filtered_tokens[u].item()])
    word2 = tokenizer.decode([filtered_tokens[v].item()])
    score = global_max_vals[i].item()
    edit_d_ratio = editdistance.eval(word1, word2) / max(len(word1), len(word2))

    #If the edit distance ratio is greater than epsilon and the score is greater than theta, add to seen pairs
    if edit_d_ratio > epsilon and score >= theta:
        pair = tuple((tuple(sorted((word1, word2))),score))
        
        if pair not in seen_pairs:
            seen_pairs.add(pair)
    


print("Number of pairs ", len(seen_pairs))
print(seen_pairs)

Number of pairs  423
{(('Lennon', 'McCartney'), 0.6115589141845703), (('Byzantines', 'Mongols'), 0.6058418154716492), (('fifteenth', 'twelfth'), 0.6070128679275513), (('Atlantic', 'Pacific'), 0.6000069379806519), (('inner', 'outer'), 0.6306939125061035), (('months', 'weeks'), 0.7051793932914734), (('therefore', 'thus'), 0.6303306221961975), (('days', 'months'), 0.6324890851974487), (('difficulties', 'problems'), 0.6333893537521362), (('Congressman', 'Senator'), 0.6088027954101562), (('due', 'owing'), 0.606035590171814), (('Fourth', 'Third'), 0.6630793809890747), (('His', 'Their'), 0.612083911895752), (('alignment', 'routing'), 0.6057812571525574), (('his', 'their'), 0.6377353668212891), (('didn', 'wasn'), 0.6333052515983582), (('brother', 'sister'), 0.6223099231719971), (('how', 'whether'), 0.6202505826950073), (('eight', 'nine'), 0.6471023559570312), (('can', 'could'), 0.6276801228523254), (('commended', 'criticized'), 0.6035046577453613), (('minutes', 'seconds'), 0.6505883932113647),

In [31]:

print(f"Number of pairs after edit distance filtering: {len(seen_pairs)}")
print(f"{'Word 1':<20} | {'Word 2':<20} | {'Cosine Sim':<10}")
print("-" * 55)
for pair in seen_pairs:
    word1 = pair[0][0]
    word2 = pair[0][1]
    score = pair[1]

    print(f"{word1:<20} | {word2:<20} | {score:.4f}")

Number of pairs after edit distance filtering: 423
Word 1               | Word 2               | Cosine Sim
-------------------------------------------------------
Lennon               | McCartney            | 0.6116
Byzantines           | Mongols              | 0.6058
fifteenth            | twelfth              | 0.6070
Atlantic             | Pacific              | 0.6000
inner                | outer                | 0.6307
months               | weeks                | 0.7052
therefore            | thus                 | 0.6303
days                 | months               | 0.6325
difficulties         | problems             | 0.6334
Congressman          | Senator              | 0.6088
due                  | owing                | 0.6060
Fourth               | Third                | 0.6631
His                  | Their                | 0.6121
alignment            | routing              | 0.6058
his                  | their                | 0.6377
didn                 | wasn              

## Step 4: Create Counts of Each Token's Occurance and Nearby Tokens

In [32]:
unique_words = dict()
unique_words_related = {}
for pair in seen_pairs:
    unique_words[pair[0][0]] = 0
    unique_words[pair[0][1]] = 0
    unique_words_related[pair[0][0]] = set()
    unique_words_related[pair[0][1]] = set()
print(f"Number of unique words in filtered pairs: {len(unique_words)}")

for pair in seen_pairs:
    unique_words[pair[0][0]] += 1
    unique_words[pair[0][1]] += 1
    unique_words_related[pair[0][0]].add(pair[0][1])
    unique_words_related[pair[0][1]].add(pair[0][0])


sorted_unique_words = sorted(unique_words.items(), key=lambda x: x[1], reverse=True)

Number of unique words in filtered pairs: 502


In [61]:
# print the words withthe highest counts

print(f"{'Word':<20} | {'Count':<10} | {'Related Words'}")
print("-" * 60)
for word, count in sorted_unique_words:
    related_words = ", ".join(sorted(unique_words_related[word]))
    print(f"{word:<20} | {count:<10} | {related_words}")

Word                 | Count      | Related Words
------------------------------------------------------------
eleventh             | 4          | fifteenth, fourteenth, thirteenth, twelfth
twelfth              | 3          | eleventh, fifteenth, thirteenth
incorrectly          | 3          | erroneously, mistakenly, wrongly
niece                | 2          | granddaughter, nephew
seventh              | 2          | ninth, sixth
four                 | 2          | five, three
three                | 2          | four, two
afternoons           | 2          | evenings, mornings
fifteenth            | 2          | eleventh, twelfth
intentionally        | 2          | deliberately, purposely
purposely            | 2          | deliberately, intentionally
hundreds             | 2          | dozens, thousands
deliberately         | 2          | intentionally, purposely
granddaughter        | 2          | grandson, niece
erroneously          | 2          | incorrectly, mistakenly
mistakenly  

## Step 5: Iterate Through Sorted Frequencies and Remove Related Words

In [ ]:
#iterate through the most common words and remove all related words

mapping = dict()

removed_words = set()
while len(sorted_unique_words) > 0:
    
    word, count = sorted_unique_words[0]
    if count == 0:
        break
    if word not in unique_words:
        continue

    #Pop the top word from the list
    unique_words.pop(word, None)
    related_words = set(unique_words_related[word])

    # Remove related words from all other entries
    for related_word in related_words:
        mapping[related_word] = word
        removed_words.add(related_word)
        #remove the keys for the related words
        unique_words.pop(related_word, None)
        unique_words_related.pop(related_word, None)
        #remove the instance of the related word from all other related word sets
        for word2 in unique_words:
            if related_word in unique_words_related[word2]:
                unique_words_related[word2].remove(related_word)
                unique_words[word2] -= 1
    sorted_unique_words = sorted(unique_words.items(), key=lambda x: x[1], reverse=True)
    

print(f"Number of removed words: {len(removed_words)}")
print(f"Removed words: {', '.join(sorted(removed_words))}")




Number of removed words: 291
Removed words: Actress, Additionally, After, AllMusic, Allmusic, Although, August, Battalion, Consequently, Conversely, December, Despite, During, Every, February, Fourth, French, GameSpot, Germans, Her, However, IGN, Immediately, Indeed, Iraq, JMA, January, July, June, Late, Liberals, Likewise, Lords, MFR, Many, March, Marge, May, McCartney, Ministry, Mongols, Moreover, NBC, NHC, NHL, Nevertheless, Nonetheless, Northumbria, Norway, November, Numerous, OCLC, October, Oxford, Pacific, Persians, Poland, Presbyterian, Prize, Regiment, Republican, Republicans, Reviewers, Rhine, Réunion, Saturday, Scheer, Second, Secretary, Senator, September, Seventh, Siber, Since, Six, Soon, Stalin, Surrey, Swedish, Telugu, Their, Though, Three, Thursday, Tintin, Trophy, Tuesday, Twitter, Upper, Various, Venezuela, When, Winnipeg, Yugoslav, autumn, battleships, before, below, bowler, bowlers, can, commended, commentators, complimented, conquered, consequently, considerably, co

In [ ]:
print(mapping)

## Step 6: Remove Words and Save Tokenizer Vocab

In [63]:
print(tokenizer.vocab_size)

model_state = tokenizer.get_vocab()
for word in removed_words:
    model_state.pop(word, None)

print(model_state)
vocab_list = [token for token, idx in sorted(model_state.items(), key=lambda x: x[1])]
print(vocab_list)
print(len(vocab_list))

import json
with open("filtered_tokenizer_vocab.json", "w", encoding="utf-8") as f:
    json.dump(vocab_list, f, ensure_ascii=False, indent=2)

30522
{'streaming': 11058, 'supervisors': 22565, 'bolts': 19947, 'pause': 8724, 'boer': 19945, 'morales': 17103, 'arenas': 26434, 'transactions': 11817, 'oath': 11292, '[unused540]': 545, 'mess': 6752, '[unused819]': 824, '##bla': 28522, '##gger': 13327, 'ripping': 17039, '##bs': 5910, 'gum': 16031, 'protein': 5250, 'stride': 18045, 'caledonia': 19305, 'halves': 23672, 'refused': 4188, 'clint': 16235, 'reworked': 27575, 'vanish': 25887, 'blending': 23293, '##fahan': 28975, 'emerged': 6003, 'hays': 29051, 'generators': 16937, 'renumbered': 27855, '##endez': 28787, 'musa': 23154, 'cuff': 26450, 'descent': 6934, '##sław': 23305, 'ordered': 3641, 'os': 9808, '##vable': 12423, 'curves': 10543, '##ina': 3981, 'embedded': 11157, 'americans': 4841, '##idal': 16975, 'shape': 4338, 'snaps': 20057, '[unused129]': 134, '##link': 13767, 'alberta': 7649, 'clicks': 29225, '##haling': 23896, 'aren': 4995, 'kathleen': 14559, 'om': 18168, 'candles': 14006, 'beast': 6841, 'drones': 24633, 'imam': 17189, 